In [ ]:
import pandas as pd
from IPython.display import display
import unicodedata

# --- Carregar Arquivo Excel ---
def carregar_excel(nome_arquivo):
    """Carrega um arquivo Excel e retorna um dicionário de abas."""
    try:
        df_dict = pd.read_excel(nome_arquivo, sheet_name=None)
        print(f"✅ Arquivo '{nome_arquivo}' carregado. Abas: {list(df_dict.keys())}")
        return df_dict
    except FileNotFoundError:
        print(f"❌ Erro: Arquivo '{nome_arquivo}' não encontrado.")
    except Exception as e:
        print(f"❌ Erro ao carregar '{nome_arquivo}': {str(e)}")
    return None

# --- Pré-processamento do DataFrame Principal ---
def processar_teste_df(teste_df):
    """Realiza transformações iniciais no DataFrame."""
    if teste_df is None or teste_df.empty:
        print("⚠️ DataFrame vazio ou inválido.")
        return None

    # Renomear colunas
    colunas_renomeadas = {
        'REF KINGS': 'Referências',
        'DESCRIÇÃO': 'Nome do Produto',
        'ATACADO': 'Valor atacado',
        'VAREJO': 'Valor varejo',
        'LINHA': 'Coleção',
        'COD. BARRAS': 'Código de Barras'
    }
    
    colunas_nao_encontradas = [k for k in colunas_renomeadas if k not in teste_df.columns]
    if colunas_nao_encontradas:
        print(f"⚠️ Colunas não encontradas: {colunas_nao_encontradas}")
    
    teste_df.rename(columns={k: v for k, v in colunas_renomeadas.items() 
        if k in teste_df.columns}, inplace=True)


    # Processar referências e adicionar colunas padrão
    if 'Referências' in teste_df.columns:
        teste_df['Código Produto - SKU'] = teste_df['Referências']
        teste_df['Referências'] = teste_df['Referências'].astype(str).str.strip().str[:-4]
        
        # Adicionar colunas constantes
        colunas_adicionais = {
            'Status do Produto': 1,
            'Exibir produto na home': 0,
            'Peso do Produto': '',
            'Tamanho do Produto': '',
            'Área de Disponibilidade': '{P}',
            'Descrição do Produto': '',
            'Grade ID': 21,
            'Imagens Vínculadas (Sim ou Não)': 'Sim',
            'Multiplicador Padrão': 1,
            'Quantidade Mínima por Pedido': 6,
            'Quantidade Máxima por Pedido': 999999,
            'Forma de Limite': 'Pela soma de todos os itens da grade'
        }
        teste_df = teste_df.assign(**colunas_adicionais)
    
    return teste_df

# --- Realizar Merges com DataFrames Auxiliares ---
def realizar_merges(teste_df, cor_df=None, fornecedores_df=None, categorias_df=None, marcas_df=None):
    """Realiza joins com os DataFrames de referência."""
    if teste_df is None:
        return None

    # Lista de merges a serem realizados
    merges = [
        (cor_df, 'COR', 'COR', 'Cores'),
        (fornecedores_df, 'FORNECEDOR', 'FORNECEDOR', 'Fornecedor ID'),
        (categorias_df, 'CATEGORIA', 'CATEGORIA', 'Categoria ID'),
        (marcas_df, 'MARCA', 'MARCA', 'Marca ID')
    ]

    for df_aux, col_teste, col_aux, nome_id in merges:
        if df_aux is None:
            print(f"⚠️ DataFrame auxiliar para {nome_id} não fornecido. Pulando merge.")
            continue
            
        try:
            # Padronização de strings
            df_aux[col_aux] = df_aux[col_aux].astype(str).apply(remover_acentos).str.upper().str.strip()
            teste_df[col_teste] = teste_df[col_teste].astype(str).apply(remover_acentos).str.upper().str.strip()
            
            # Merge e tratamento pós-join
            antes = len(teste_df)
            teste_df = teste_df.merge(
                df_aux[[col_aux, 'ID']], 
                left_on=col_teste, 
                right_on=col_aux, 
                how='left'
            ).rename(columns={'ID': nome_id})
            
            # Verificar resultados do merge
            sem_match = teste_df[nome_id].isna().sum()
            print(f"🔗 Merge com {nome_id}: {len(teste_df)-antes} linhas adicionadas | {sem_match} sem correspondência")
            
            teste_df.drop(columns=[col_aux], inplace=True, errors='ignore')
            
        except KeyError as e:
            print(f"❌ Erro no merge com {nome_id}: {str(e)}")
    
    return teste_df

# --- Organizar Colunas e Remover Duplicatas ---
def organizar_colunas(teste_df):
    """Padroniza a estrutura final do DataFrame com a ordem de colunas especificada."""
    if teste_df is None:
        return None

    # Verificar e remover duplicatas
    duplicatas = teste_df.duplicated(subset=['Código Produto - SKU'], keep=False)
    if duplicatas.any():
        print(f"⚠️ Removendo {duplicatas.sum()} duplicatas de 'Código Produto - SKU'")
        teste_df = teste_df.drop_duplicates(subset=['Código Produto - SKU'])

    # Ordem EXATA das colunas conforme solicitado
    colunas_ordenadas = [
        'Referências', 'Código Produto - SKU', 'Status do Produto', 'Nome do Produto', 
        'Categoria ID', 'Grade ID', 'Exibir produto na home', 'Valor atacado', 
        'Peso do Produto', 'Tamanho do Produto', 'Marca ID', 'Fornecedor ID', 
        'Código de Barras', 'Coleção', 'Valor varejo', 'Área de Disponibilidade',
        'Descrição do Produto', 'Cores', 'Imagens Vínculadas (Sim ou Não)', 
        'Multiplicador Padrão', 'Quantidade Mínima por Pedido', 
        'Quantidade Máxima por Pedido', 'Forma de Limite'
    ]

    # Verificar se todas as colunas existem (apenas para log)
    colunas_faltantes = [col for col in colunas_ordenadas if col not in teste_df.columns]
    if colunas_faltantes:
        print(f"⚠️ ATENÇÃO: Colunas faltantes no DataFrame: {colunas_faltantes}")

    # Retorna apenas as colunas existentes, na ordem solicitada
    return teste_df[[col for col in colunas_ordenadas if col in teste_df.columns]]
# --- Salvar com Formatação Numérica ---
def salvar_excel(df, nome_arquivo):
    """Salva o DataFrame garantindo formatação numérica."""
    if df is None:
        return

    try:
        # Garantir formatação antes de salvar
        colunas_numericas = ['Valor atacado', 'Valor varejo']
        for col in colunas_numericas:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: f"{float(x):.2f}" if pd.notnull(x) else "")

        # Salvar arquivo
        df.to_excel(nome_arquivo, index=False, engine='openpyxl')
        print(f"💾 Arquivo salvo como '{nome_arquivo}' com formatação numérica preservada.")
        
    except PermissionError:
        print(f"❌ Erro: Permissão negada para salvar '{nome_arquivo}'")
    except Exception as e:
        print(f"❌ Erro ao salvar: {str(e)}")

# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    # Carregar dados
    print("\n" + "="*50)
    print("CARREGANDO ARQUIVOS...")
    teste_sheets = carregar_excel("teste.xlsx")
    
    if not teste_sheets:
        print("❌ Nenhum dado carregado. Verifique os arquivos de entrada.")
    else:
        # Carregar DataFrames auxiliares
        cor_df = carregar_excel("cor.xlsx").get('Planilha1') if carregar_excel("cor.xlsx") else None
        categorias_df = carregar_excel("categorias.xlsx").get('Planilha1') if carregar_excel("categorias.xlsx") else None
        marcas_df = carregar_excel("marcas.xlsx").get('Planilha1') if carregar_excel("marcas.xlsx") else None
        fornecedores_df = carregar_excel("fornecedores.xlsx").get('Planilha1') if carregar_excel("fornecedores.xlsx") else None

        # Processar cada aba
        for nome_aba, teste_df in teste_sheets.items():
            print("\n" + "="*50)
            print(f"📊 PROCESSANDO ABA: {nome_aba}")
            
            # Pipeline de processamento
            teste_df = processar_teste_df(teste_df)
            teste_df = realizar_merges(teste_df, cor_df, fornecedores_df, categorias_df, marcas_df)
            teste_df = organizar_colunas(teste_df)
            
            if teste_df is not None:
                # Visualização e salvamento
                print("\n🔍 Amostra dos dados processados:")
                display(teste_df.head(3))
                
                nome_saida = f"resultado_processado_{nome_aba}.xlsx"
                salvar_excel(teste_df, nome_saida)


CARREGANDO ARQUIVOS...
✅ Arquivo 'teste.xlsx' carregado. Abas: ['TOY STORY', 'MEMÓRIAS', 'TECH BRUNX DL 1', 'MILIONÁRIOS BRNX DEL 1', 'STITCH', 'TECH BRUNX DL2', 'FEMININO DL 1', 'MILIONÁRIOS DEEDS DL1', 'CULTURA POPULAR BRASILEIRA', 'O ESTRANHO MUNDO DE JACK', 'GOLF CLUB', 'GOLF CLUB BRUNX ', 'ESSENTIALS', 'FEMININO DL 2', 'MILIONÁRIOS DEEDS DEL 2', 'MILIONÁRIOS BRNX DEL 2', 'BASIC PACK']
✅ Arquivo 'cor.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'cor.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'categorias.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'categorias.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'marcas.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'marcas.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'fornecedores.xlsx' carregado. Abas: ['Planilha1']
✅ Arquivo 'fornecedores.xlsx' carregado. Abas: ['Planilha1']

📊 PROCESSANDO ABA: TOY STORY
🔗 Merge com Cores: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Fornecedor ID: 0 linhas adicionadas | 0 sem co

,Referências,Código Produto - SKU,Status do Produto,Nome do Produto,Categoria ID,Grade ID,Exibir produto na home,Valor atacado,Peso do Produto,Tamanho do Produto,...,Coleção,Valor varejo,Área de Disponibilidade,Descrição do Produto,Cores,Imagens Vínculadas (Sim ou Não),Multiplicador Padrão,Quantidade Mínima por Pedido,Quantidade Máxima por Pedido,Forma de Limite
0,CAM1VER26TOY,CAM1VER26TOY-001,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,0,70.56,,,...,KINGS COLLAB PIXAR TOY STORY,179.99,{P},,1,Sim,1,6,999999,Pela soma de todos os itens da grade
4,CAM1VER26TOY,CAM1VER26TOY-006,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,0,70.56,,,...,KINGS COLLAB PIXAR TOY STORY,179.99,{P},,2,Sim,1,6,999999,Pela soma de todos os itens da grade
8,CAM2VER26TOY,CAM2VER26TOY-001,1,Camiseta oversized Kings Collab pixar Toy Stor...,3,21,0,70.56,,,...,KINGS COLLAB PIXAR TOY STORY,179.99,{P},,1,Sim,1,6,999999,Pela soma de todos os itens da grade


💾 Arquivo salvo como 'resultado_processado_TOY STORY.xlsx' com formatação numérica preservada.

📊 PROCESSANDO ABA: MEMÓRIAS
🔗 Merge com Cores: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Fornecedor ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Categoria ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Marca ID: 0 linhas adicionadas | 0 sem correspondência
⚠️ Removendo 80 duplicatas de 'Código Produto - SKU'

🔍 Amostra dos dados processados:


,Referências,Código Produto - SKU,Status do Produto,Nome do Produto,Categoria ID,Grade ID,Exibir produto na home,Valor atacado,Peso do Produto,Tamanho do Produto,...,Coleção,Valor varejo,Área de Disponibilidade,Descrição do Produto,Cores,Imagens Vínculadas (Sim ou Não),Multiplicador Padrão,Quantidade Mínima por Pedido,Quantidade Máxima por Pedido,Forma de Limite
0,CMT7VER26,CMT7VER26-001,1,Camiseta oversized Kings memórias amor sem fim,3,21,0,71.996,,,...,MEMÓRIAS,179.99,{P},,1,Sim,1,6,999999,Pela soma de todos os itens da grade
4,CMT7VER26,CMT7VER26-006,1,Camiseta oversized Kings memórias amor sem fim,3,21,0,71.996,,,...,MEMÓRIAS,179.99,{P},,2,Sim,1,6,999999,Pela soma de todos os itens da grade
8,CMT7VER26,CMT7VER26-016,1,Camiseta oversized Kings memórias amor sem fim,3,21,0,71.996,,,...,MEMÓRIAS,179.99,{P},,15,Sim,1,6,999999,Pela soma de todos os itens da grade


💾 Arquivo salvo como 'resultado_processado_MEMÓRIAS.xlsx' com formatação numérica preservada.

📊 PROCESSANDO ABA: TECH BRUNX DL 1
🔗 Merge com Cores: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Fornecedor ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Categoria ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Marca ID: 0 linhas adicionadas | 0 sem correspondência
⚠️ Removendo 44 duplicatas de 'Código Produto - SKU'

🔍 Amostra dos dados processados:


...

💾 Arquivo salvo como 'resultado_processado_MILIONÁRIOS DEEDS DEL 2.xlsx' com formatação numérica preservada.

📊 PROCESSANDO ABA: MILIONÁRIOS BRNX DEL 2
🔗 Merge com Cores: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Fornecedor ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Categoria ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Marca ID: 0 linhas adicionadas | 0 sem correspondência
⚠️ Removendo 140 duplicatas de 'Código Produto - SKU'

🔍 Amostra dos dados processados:


,Referências,Código Produto - SKU,Status do Produto,Nome do Produto,Categoria ID,Grade ID,Exibir produto na home,Valor atacado,Peso do Produto,Tamanho do Produto,...,Coleção,Valor varejo,Área de Disponibilidade,Descrição do Produto,Cores,Imagens Vínculadas (Sim ou Não),Multiplicador Padrão,Quantidade Mínima por Pedido,Quantidade Máxima por Pedido,Forma de Limite
0,CMT13VER26MLRS,CMT13VER26MLRS-006,1,Camiseta Milionários Logo,3,21,0,67.0,,,...,MILIONÁRIOS,189.99,{P},,2,Sim,1,6,999999,Pela soma de todos os itens da grade
4,CMT13VER26MLRS,CMT13VER26MLRS-001,1,Camiseta Milionários Logo,3,21,0,67.0,,,...,MILIONÁRIOS,189.99,{P},,1,Sim,1,6,999999,Pela soma de todos os itens da grade
8,CMT14VER26MLRS,CMT14VER26MLRS-002,1,Camiseta Milionários monograma,3,21,0,62.0,,,...,MILIONÁRIOS,179.99,{P},,7,Sim,1,6,999999,Pela soma de todos os itens da grade


💾 Arquivo salvo como 'resultado_processado_MILIONÁRIOS BRNX DEL 2.xlsx' com formatação numérica preservada.

📊 PROCESSANDO ABA: BASIC PACK
🔗 Merge com Cores: 0 linhas adicionadas | 4 sem correspondência
🔗 Merge com Fornecedor ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Categoria ID: 0 linhas adicionadas | 0 sem correspondência
🔗 Merge com Marca ID: 0 linhas adicionadas | 0 sem correspondência
⚠️ Removendo 14 duplicatas de 'Código Produto - SKU'

🔍 Amostra dos dados processados:


,Referências,Código Produto - SKU,Status do Produto,Nome do Produto,Categoria ID,Grade ID,Exibir produto na home,Valor atacado,Peso do Produto,Tamanho do Produto,...,Coleção,Valor varejo,Área de Disponibilidade,Descrição do Produto,Cores,Imagens Vínculadas (Sim ou Não),Multiplicador Padrão,Quantidade Mínima por Pedido,Quantidade Máxima por Pedido,Forma de Limite
0,UNDER-25,UNDER-25KGS1,1,KIT 3 CUECAS BOXER BRANCAS,39,21,0,31.996,,,...,UNDERWEAR,79.99,{P},,2,Sim,1,6,999999,Pela soma de todos os itens da grade
5,UNDER-25,UNDER-25KGS2,1,KIT 3 CUECAS BOXER PRETOS,39,21,0,31.996,,,...,UNDERWEAR,79.99,{P},,1,Sim,1,6,999999,Pela soma de todos os itens da grade
10,UNDER-24,UNDER-24KGS2,1,KIT COM 3 CAMISETAS LETTERING LOGO,3,21,0,87.996,,,...,BASIC PACK,219.99,{P},,NaN,Sim,1,6,999999,Pela soma de todos os itens da grade


💾 Arquivo salvo como 'resultado_processado_BASIC PACK.xlsx' com formatação numérica preservada.


In [81]:
teste_df.dtypes

Referências                        object
Código Produto - SKU               object
Status do Produto                   int64
Nome do Produto                    object
Categoria ID                        int64
Grade ID                            int64
Exibir produto na home              int64
Valor atacado                      object
Peso do Produto                    object
Tamanho do Produto                 object
Marca ID                           object
Fornecedor ID                      object
Código de Barras                    int64
Coleção                            object
Valor varejo                       object
Área de Disponibilidade            object
Descrição do Produto               object
Cores                              object
Imagens Vínculadas (Sim ou Não)    object
Multiplicador Padrão                int64
Quantidade Mínima por Pedido        int64
Quantidade Máxima por Pedido        int64
Forma de Limite                    object
dtype: object